In [3]:
#!pip install praw


In [4]:
import praw
import pandas as pd
from datetime import datetime, timedelta

def get_reliable_real_madrid_sentiment():
    """
    Obtener sentimiento MÁS FIABLE de fans del Real Madrid
    """
    
    # Reddit API (gratuita)
    reddit = praw.Reddit(
        client_id='your_client_id',
        client_secret='your_secret',
        user_agent='real_madrid_sentiment_analyzer'
    )
    
    # r/realmadrid - la fuente MÁS FIABLE
    subreddit = reddit.subreddit('realmadrid')
    
    reliable_data = []
    
    # Posts recientes con engagement alto (filtro de calidad)
    for post in subreddit.hot(limit=50):
        
        # Filtros de fiabilidad
        if post.score > 10 and post.num_comments > 5:  # Engagement mínimo
            
            post_data = {
                'title': post.title,
                'author': post.author.name if post.author else 'deleted',
                'score': post.score,
                'upvote_ratio': post.upvote_ratio,
                'num_comments': post.num_comments,
                'created': datetime.fromtimestamp(post.created_utc),
                'url': post.url,
                'selftext': post.selftext,
                'author_karma': post.author.comment_karma if post.author else 0,
                'reliability_score': calculate_reliability(post)
            }
            
            # Comentarios de calidad
            post.comments.replace_more(limit=5)
            comments = []
            
            for comment in post.comments.list()[:20]:
                if comment.score > 2:  # Solo comentarios con upvotes
                    comments.append({
                        'text': comment.body,
                        'score': comment.score,
                        'author_karma': comment.author.comment_karma if comment.author else 0
                    })
            
            post_data['top_comments'] = comments
            reliable_data.append(post_data)
    
    return reliable_data

def calculate_reliability(post):
    """
    Calcular score de fiabilidad basado en métricas Reddit
    """
    
    # Factores de fiabilidad
    score_factor = min(post.score / 100, 1.0)  # Normalizado
    ratio_factor = post.upvote_ratio  # Ratio positivo
    comment_factor = min(post.num_comments / 50, 1.0)  # Engagement
    author_factor = 0.1 if post.author and post.author.comment_karma > 1000 else 0
    
    reliability = (score_factor * 0.3 + 
                  ratio_factor * 0.4 + 
                  comment_factor * 0.2 + 
                  author_factor * 0.1)
    
    return round(reliability, 2)